# LSTM auto-encoder with bidirectional encoder and seq2seq decoder with attention

In [2]:
train = False


import numpy as np 
import tensorflow as tf
import helpers #for formatting data into batches and generating random sequence data

tf.reset_default_graph() #Clears the default graph stack and resets the global default graph.
sess = tf.InteractiveSession() #initializes a tensorflow session

In [3]:
PAD = 0 # padding
BOS = 1 # Begin of sequence
EOS = 2 # End of sequence

vocab_size = 10 # max length of input sequence
input_embedding_size = 20 # character length (vector)

encoder_hidden_units = 16 # num neurons
decoder_hidden_units = encoder_hidden_units * 2

In [4]:
#input placehodlers
encoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='encoder_inputs')
#contains the lengths for each of the sequence in the batch, we will pad so all the same
#if you don't want to pad, check out dynamic memory networks to input variable length sequences
encoder_inputs_length = tf.placeholder(shape=(None,), dtype=tf.int32, name='encoder_inputs_length')
decoder_targets = tf.placeholder(shape=(None, None), dtype=tf.int32, name='decoder_targets')

In [5]:
embeddings = tf.Variable(tf.random_uniform([vocab_size, input_embedding_size], -1.0, 1.0), dtype=tf.float32)

#this thing could get huge in a real world application
encoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, encoder_inputs)

# Encoder
Bidirectional encoder

In [6]:
from tensorflow.python.ops.rnn_cell import LSTMCell, LSTMStateTuple

In [7]:
encoder_cell =  LSTMCell(encoder_hidden_units)
#encoder_cell = tf.contrib.rnn.DropoutWrapper(encoder_cell, output_keep_prob=0.5, input_keep_prob=1.0, state_keep_prob=1.0)

In [8]:
((encoder_fw_outputs,
  encoder_bw_outputs),
 (encoder_fw_final_state,
  encoder_bw_final_state)) = (
    tf.nn.bidirectional_dynamic_rnn(cell_fw=encoder_cell,
                                    cell_bw=encoder_cell,
                                    inputs=encoder_inputs_embedded,
                                    sequence_length=encoder_inputs_length,
                                    dtype=tf.float32, time_major=True)
    )

In [9]:
#Concatenates tensors along one dimension.
encoder_outputs = tf.concat((encoder_fw_outputs, encoder_bw_outputs), 2)

#letters h and c are commonly used to denote "output value" and "cell state". 
#http://colah.github.io/posts/2015-08-Understanding-LSTMs/ 
#Those tensors represent combined internal state of the cell, and should be passed together. 

encoder_final_state_c = tf.concat(
    (encoder_fw_final_state.c, encoder_bw_final_state.c), 1)

encoder_final_state_h = tf.concat(
    (encoder_fw_final_state.h, encoder_bw_final_state.h), 1)

#TF Tuple used by LSTM Cells for state_size, zero_state, and output state.
encoder_final_state = LSTMStateTuple(
    c=encoder_final_state_c,
    h=encoder_final_state_h
)

# Decoder
Common part

In [10]:
decoder_cell = LSTMCell(decoder_hidden_units)

In [11]:
encoder_max_time, batch_size = tf.unstack(tf.shape(encoder_inputs))

In [12]:
decoder_lengths = encoder_inputs_length + 2
# +2 additional steps (PAD token)

In [13]:
max_length = encoder_max_time + 2

decoder_inputs = tf.ones([max_length, batch_size], dtype=tf.int32)
decoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, decoder_inputs)
decoder_inputs_embedded

<tf.Tensor 'embedding_lookup_1:0' shape=(?, ?, 20) dtype=float32>

## Dynamic_rnn decoder
For comparison purpose (no attention mechanisms)

You can jump direclty to the seq2seq decoder part without running those cells

In [14]:
#((decoder_outputs),
# (decoder_state)) = (
#    tf.nn.dynamic_rnn(cell=decoder_cell,
#                      inputs=decoder_inputs_embedded, # Make sense for training but how to infer?
#                      sequence_length=decoder_lengths,
#                      initial_state=encoder_final_state,
#                      dtype=tf.float32, time_major=True)
#    )

Output projection

In [15]:
#manually specifying since we are going to implement attention details for the decoder in a sec
#weights
#W = tf.Variable(tf.random_uniform([decoder_hidden_units, vocab_size], -1, 1), dtype=tf.float32)
#bias
#b = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)

In [16]:
#to convert output to human readable prediction
#we will reshape output tensor

#Unpacks the given dimension of a rank-R tensor into rank-(R-1) tensors.
#reduces dimensionality
#decoder_max_steps, decoder_batch_size, decoder_dim = tf.unstack(tf.shape(decoder_outputs))
#flettened output tensor
#decoder_outputs_flat = tf.reshape(decoder_outputs, (-1, decoder_dim))
#pass flattened tensor through decoder
#decoder_logits_flat = tf.add(tf.matmul(decoder_outputs_flat, W), b)
#prediction vals
#decoder_logits = tf.reshape(decoder_logits_flat, (decoder_max_steps, decoder_batch_size, vocab_size))

In [17]:
#final prediction
#decoder_prediction = tf.argmax(decoder_logits, 2)

## Seq2seq decoder

In [18]:
from tensorflow.python.layers import core as layers_core

In [19]:
# attention_states: [batch_size, max_time, num_units]
attention_states = tf.transpose(encoder_outputs, [1, 0, 2])

# Create an attention mechanism
attention_mechanism = tf.contrib.seq2seq.LuongAttention(
    num_units=decoder_hidden_units, 
    memory=attention_states,
    #memory_sequence_length=None # default value
)

decoder_cell = tf.contrib.seq2seq.AttentionWrapper(
    cell=decoder_cell, 
    attention_mechanism=attention_mechanism,
    attention_layer_size=decoder_hidden_units)

attention_zero = decoder_cell.zero_state(batch_size=batch_size, dtype=tf.float32)
encoder_final_state_attention = attention_zero.clone(cell_state=encoder_final_state)

In [20]:
projection_layer = layers_core.Dense(vocab_size, use_bias=True)

# Helper
if train:
    helper = tf.contrib.seq2seq.TrainingHelper(
        inputs=decoder_inputs_embedded, 
        sequence_length=decoder_lengths, 
        time_major=True)

else:
    helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
          embedding=embeddings,
          start_tokens=tf.tile([BOS], [batch_size]),
          end_token=EOS)
    print(batch_size)
    print(helper)

# Decoder
decoder = tf.contrib.seq2seq.BasicDecoder(
    cell=decoder_cell, 
    helper=helper, 
    initial_state=encoder_final_state_attention, # to use without attention change to 'encoder_final_state'
    output_layer=projection_layer)
print(decoder)

# Dynamic decoding
(decoder_outputs, final_state, final_sequence_lengths) = tf.contrib.seq2seq.dynamic_decode(
    decoder,
    output_time_major=True
)
decoder_logits = decoder_outputs.rnn_output
decoder_prediction = decoder_outputs.sample_id
print(decoder_prediction)

Tensor("unstack:1", shape=(), dtype=int32)
Tensor("decoder/TensorArrayStack_1/TensorArrayGatherV3:0", shape=(?, ?), dtype=int32)


# Optimizer

In [21]:
if train:
    #cross entropy loss
    #one hot encode the target values so we don't rank just differentiate
    stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.one_hot(decoder_targets, depth=vocab_size, dtype=tf.float32),
        logits=decoder_logits,
    )

    #loss function
    loss = tf.reduce_mean(stepwise_cross_entropy)
    #train it 
    train_op = tf.train.AdamOptimizer().minimize(loss)

In [22]:
if train:
    sess.run(tf.global_variables_initializer())

# Load toy dataset

In [23]:
batch_size = 128 if train else 1

batches = helpers.random_sequences(length_from=3, length_to=8,
                                   vocab_lower=3, vocab_upper=10,
                                   batch_size=batch_size)

print('Head of the batch:')
for seq in next(batches)[:10]:
    print(seq)

Head of the batch:
[8, 6, 7, 9]


In [24]:
def next_feed():
    batch = next(batches)
    encoder_inputs_, encoder_input_lengths_ = helpers.batch(
        [[BOS] + (sequence) + [EOS] for sequence in batch]
    )
    decoder_targets_, _ = helpers.batch(
        [[BOS] + (sequence) + [EOS] + [PAD] * 2 for sequence in batch]
    )
    return {
        encoder_inputs: encoder_inputs_,
        encoder_inputs_length: encoder_input_lengths_,
        decoder_targets: decoder_targets_,
    }

In [25]:
loss_track = []

In [26]:
def train_model(max_batches=1001, batches_in_epoch=500):
    try:
        for batch in range(max_batches):
            fd = next_feed()
            _, l = sess.run([train_op, loss], fd)
            loss_track.append(l)

            if batch == 0 or batch % batches_in_epoch == 0:
                predict_, l = sess.run([decoder_prediction, loss], fd)
                print('batch {}'.format(batch))
                print('  minibatch loss: {}'.format(l))
                for i, (inp, pred) in enumerate(zip(fd[encoder_inputs].T, predict_.T)):
                    print('  sample {}:'.format(i + 1))
                    print('    input     > {}'.format(inp))
                    print('    predicted > {}'.format(pred))
                    if i >= 2:
                        break
                print

    except KeyboardInterrupt:
        print('training interrupted')

# Train the model
Possible to skip those cells and load a model checkpoint

In [27]:
if train:
    train_model()

Note: Remember that '1' is the BOS tag and '2' the EOS tag, not predicted outputs!

In [28]:
if train:
    %matplotlib inline
    import matplotlib.pyplot as plt
    plt.yscale('log')
    plt.plot(loss_track)
    print('loss {:.4f} after {} examples (batch_size={})'.format(loss_track[-1], len(loss_track)*batch_size, batch_size))

Save the model (create a checkpoint under the 'checkpoints' directory

In [29]:
if train:
    saver = tf.train.Saver()
    saver.save(sess, "checkpoints/seq2seq.ckpt")

# Load a checkpoint

In [30]:
if not train:
    saver = tf.train.Saver()
    saver.restore(sess, "checkpoints/seq2seq.ckpt")

INFO:tensorflow:Restoring parameters from checkpoints/seq2seq.ckpt


# Inference

In [31]:
if not train:
    fd = next_feed()
    fd.pop(decoder_targets, None)
    print(fd.keys())

dict_keys([<tf.Tensor 'encoder_inputs:0' shape=(?, ?) dtype=int32>, <tf.Tensor 'encoder_inputs_length:0' shape=(?,) dtype=int32>])


In [32]:
if not train:
    # Just a simple test
    fd = next_feed()
    fd.pop(decoder_targets, None)

    batch, l = fd[encoder_inputs], fd[encoder_inputs_length]
    print(batch.shape)
    print(l)
    print(batch)
    print(decoder_prediction)
    print(fd)

    pred = sess.run(decoder_prediction, fd)
    f = fd[encoder_inputs].T

    print(f[0])
    print(pred.T[0])

(9, 1)
[9]
[[1]
 [9]
 [3]
 [7]
 [7]
 [8]
 [4]
 [6]
 [2]]
Tensor("decoder/TensorArrayStack_1/TensorArrayGatherV3:0", shape=(?, ?), dtype=int32)
{<tf.Tensor 'encoder_inputs:0' shape=(?, ?) dtype=int32>: array([[1],
       [9],
       [3],
       [7],
       [7],
       [8],
       [4],
       [6],
       [2]], dtype=int32), <tf.Tensor 'encoder_inputs_length:0' shape=(?,) dtype=int32>: [9]}
[1 9 3 7 7 8 4 6 2]
[1 9 3 7 7 7 8 4 4 6 2]


In [33]:
if not train:
    # Simple reconstruction accuracy
    import difflib

    def reconstruction_accurary(target, prediction):
        target = "".join(str(x) for x in target[1:-1].tolist()) # Discard BOS and EOS
        prediction = "".join(str(x) for x in prediction[1:-1].tolist())
        seq = difflib.SequenceMatcher(None, target, prediction)
        return seq.ratio()*100
        print('Reconstruction accuracy {:.4f} %'.format(seq.ratio()*100))

    #print('Reconstruction accuracy {:.4f} %'.format(reconstruction_accurary(f[0], pred.T[0])))

In [ ]:
if not train:
    fds, enc_outs, dec_embs, preds = [], [], [], []

    for i in range(1000):
        print(i)
        fd = next_feed()
        fd.pop(decoder_targets, None)
        fds.append(fd[encoder_inputs].T[0])

        pred = sess.run(decoder_prediction, fd)
        preds.append(pred.T[0])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [34]:
if not train:
    for i in range(len(preds)):
        print("Sample " + str(i) + ":")
        print(fds[i])
        print(preds[i])
        print('Reconstruction accuracy {:.4f} %'.format(reconstruction_accurary(fds[i], preds[i])))
        print

Sample 0:
[1 5 8 3 2]
[1 5 8 3 2]
Reconstruction accuracy 100.0000 %
Sample 1:
[1 6 3 5 3 2]
[1 6 3 5 5 3 2]
Reconstruction accuracy 88.8889 %
Sample 2:
[1 9 6 6 9 4 9 2]
[1 9 6 6 9 9 4 9 2]
Reconstruction accuracy 92.3077 %
Sample 3:
[1 9 3 7 3 9 3 3 5 2]
[1 9 3 7 7 3 9 9 9 3 3 3 3 3 5 2]
Reconstruction accuracy 72.7273 %
Sample 4:
[1 3 9 9 2]
[1 3 9 9 9 2]
Reconstruction accuracy 85.7143 %
Sample 5:
[1 7 5 7 3 3 8 2]
[1 7 5 7 7 3 3 3 8 2]
Reconstruction accuracy 85.7143 %
Sample 6:
[1 3 7 4 5 2]
[1 3 7 4 5 5 2]
Reconstruction accuracy 88.8889 %
Sample 7:
[1 6 5 3 3 2]
[1 6 5 3 3 3 2]
Reconstruction accuracy 88.8889 %
Sample 8:
[1 6 5 8 8 4 6 3 2]
[1 6 5 8 8 8 4 4 6 3 2]
Reconstruction accuracy 87.5000 %
Sample 9:
[1 4 8 8 4 3 3 2]
[1 4 8 8 8 4 3 3 3 2]
Reconstruction accuracy 85.7143 %


In [39]:
decoder_prediction

<tf.Tensor 'decoder/TensorArrayStack_1/TensorArrayGatherV3:0' shape=(?, ?) dtype=int32>

# Visualize graph

Copy from https://stackoverflow.com/questions/38189119/simple-way-to-visualize-a-tensorflow-graph-in-jupyter

In [35]:
if 0:
    from IPython.display import clear_output, Image, display, HTML

    def strip_consts(graph_def, max_const_size=32):
        """Strip large constant values from graph_def."""
        strip_def = tf.GraphDef()
        for n0 in graph_def.node:
            n = strip_def.node.add() 
            n.MergeFrom(n0)
            if n.op == 'Const':
                tensor = n.attr['value'].tensor
                size = len(tensor.tensor_content)
                if size > max_const_size:
                    tensor.tensor_content = "<stripped %d bytes>"%size
        return strip_def

    def show_graph(graph_def, max_const_size=32):
        """Visualize TensorFlow graph."""
        if hasattr(graph_def, 'as_graph_def'):
            graph_def = graph_def.as_graph_def()
        strip_def = strip_consts(graph_def, max_const_size=max_const_size)
        code = """
            <script>
              function load() {{
                document.getElementById("{id}").pbtxt = {data};
              }}
            </script>
            <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
            <div style="height:600px">
              <tf-graph-basic id="{id}"></tf-graph-basic>
            </div>
        """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

        iframe = """
            <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
        """.format(code.replace('"', '&quot;'))
        display(HTML(iframe))
        
    show_graph(tf.get_default_graph().as_graph_def())